In [1]:
import numpy as np 
import networkx as nx
from scipy.sparse import coo_matrix

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator
from qiskit.circuit.library import TwoLocal
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit import transpile, QuantumCircuit
from qiskit_optimization.translators import to_docplex_mp
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp


import optimizer as opt

### Load original problem

In [2]:
with open('../dynamic_reinitialization_algorithm/instances/MS/MS_6_50_3.npy','rb') as f:
    A = np.load(f).astype(int).T
    b = np.load(f).astype(int)

### Get adjacacency matrix for the QUBO problem

In [3]:
adj = A.T@A -A.T@b - b.T@A  
offset = b.T@b

### Convert to Ising

In [4]:
J_ising = np.triu(adj).copy()
J_ising = J_ising/4.0

h_ising = np.zeros(adj.shape[0])
for i in range(0, adj.shape[0]):
    h_ising[i] = 0.25 * np.sum(adj[i,:]) + 0.25 * np.sum(adj[i,i])
    
offset_ising = (0.5 * np.sum([np.triu(adj)[i,i] for i in range(0, adj.shape[0])]) 
+ 0.125 * np.sum([(np.sum(adj[:, i]) - adj[i,i]) for i in range(0, adj.shape[0])]))

### Do gauge transformation

In [5]:
def adapt_H(J, h, x):
    
    J_tr = J.copy()
    h_tr = h.copy()
    
    for i in range(0, J_tr.shape[0]):
        if x[i] == 1.0:
            h_tr[i]*=(-1)
            
    for idx, _ in np.ndenumerate(J_tr):
        if idx[0] != idx[1]:
            J_tr[idx[0], idx[1]]*=(-1) ** (x[idx[0]] + x[idx[1]])
            
    return J_tr, h_tr

In [19]:
num_qubits = adj.shape[0]

x_tr = np.random.choice([1, 0], size=num_qubits, p=[0.0, 1.0])
J_tr, h_tr = adapt_H(J_ising, h_ising, x_tr)

In [20]:
x_tr

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [21]:
def objective(x, J, h, offset):
    return x @ J @ x + np.dot(h.T, x) + offset

### Run VQA

In [22]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_aer.primitives import Sampler
from qiskit_aer.backends import AerSimulator
from qiskit.circuit.library import TwoLocal
#from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Sampler, Session, Options
from qiskit import transpile, QuantumCircuit
from qiskit_optimization.translators import to_docplex_mp
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader
from qiskit_optimization.translators import from_docplex_mp
from qiskit_aer.primitives import Sampler

import optimizer as opt

reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    reps = reps,
    insert_barriers = True
    )

bin_init = np.random.choice([1, 0], size=num_qubits, p=[0.9, 0.1])
theta_initial = np.hstack([np.full(num_qubits, np.pi/2)*bin_init, np.zeros(num_qubits*ansatz.reps)])


n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

shots = 2**15

sampler = Sampler(backend_options={'method' : 'matrix_product_state', 'n_qubits': num_qubits})
sampler.set_options(shots=shots)
#theta = QuantumOptimizer.theta_log[239]

QuantumOptimizer = opt.Quantum_Optimizer()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler, objective, J_tr, h_tr, offset + offset_ising)

run = QuantumOptimizer.run(
    objective = objective,
    qc_params_init = theta_initial,
    random_update = False, 
    max_epoch = 2,  
    intermediate_result_display = True,
    refval = None,
    alpha = 0.1 #cVAR param
)

number of params: 150
epoch = 0 	 step = 0 	 best val = -497146806.2 	 expectation = -276718680.0
epoch = 0 	 step = 1 	 best val = -497146806.2 	 expectation = -299303982.5
epoch = 0 	 step = 2 	 best val = -497146806.2 	 expectation = -320288356.9
epoch = 0 	 step = 3 	 best val = -500306954.2 	 expectation = -327812133.1
epoch = 0 	 step = 4 	 best val = -553726311.2 	 expectation = -337508838.5
epoch = 0 	 step = 5 	 best val = -553726311.2 	 expectation = -346257489.3
epoch = 0 	 step = 6 	 best val = -553726311.2 	 expectation = -356454367.0
epoch = 0 	 step = 7 	 best val = -553726311.2 	 expectation = -366238106.4
epoch = 0 	 step = 8 	 best val = -577929439.2 	 expectation = -377065588.5
epoch = 0 	 step = 9 	 best val = -581459244.2 	 expectation = -384215815.3
epoch = 0 	 step = 10 	 best val = -581459244.2 	 expectation = -394769863.3
epoch = 0 	 step = 11 	 best val = -608987978.2 	 expectation = -402823872.0
epoch = 0 	 step = 12 	 best val = -632886612.2 	 expectation = 

In [23]:
best_x = QuantumOptimizer.best_x

In [24]:
best_x

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [16]:
x_tr

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0])

In [17]:
best_x_trans = []
for i,j in zip(best_x, x_tr):
    if j == 1:
        best_x_trans.append(1-i)
    else:
        best_x_trans.append(i)
        
best_x_trans

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [18]:
best_x

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1])